# Tutorial of hdfs python library

References:
- [PyHive Documentation](https://github.com/dropbox/PyHive)
- [Using Hive External Tables](https://docs.cloudera.com/HDPDocuments/HDP3/HDP-3.1.5/using-hiveql/content/hive_create_an_external_table.html)

Install hdfs library from pyPi repo:

In [ ]:
!pip install --user pyhive

References:
- [PyHive Documentation](https://github.com/dropbox/PyHive)
- [Using Hive External Tables](https://docs.cloudera.com/HDPDocuments/HDP3/HDP-3.1.5/using-hiveql/content/hive_create_an_external_table.html)

In [ ]:
!pip show parquet

## HDFS DATASET
Subimos el dataset a hdfs

In [2]:
from hdfs import InsecureClient
hdfs_client = InsecureClient('http://hive-hdfs:50070', user='hdfs')
if not(hdfs_client.status('/datasets/world_cities', strict=False)):
    # Download a file or folder locally.
    hdfs_client.makedirs('/datasets/world_cities')
    hdfs_client.upload('/datasets/world_cities','world-cities_csv.csv', n_threads=5)

hdfs_client.list('/datasets/world_cities',status=True)

[('world-cities_csv.csv',
  {'accessTime': 1613144968999,
   'blockSize': 134217728,
   'childrenNum': 0,
   'fileId': 16396,
   'group': 'supergroup',
   'length': 895586,
   'modificationTime': 1613144969870,
   'owner': 'hdfs',
   'pathSuffix': 'world-cities_csv.csv',
   'permission': '755',
   'replication': 3,
   'storagePolicy': 0,
   'type': 'FILE'})]

In [4]:
from pyhive import hive
conn = hive.Connection(host="hive-server", port=10000, username="hive")
cursor = conn.cursor()

## Consultas SQL con Hive

Directamente sobre los archivos csv almacenados en hdfs

Crear base de datos de prueba en el metastore

In [5]:
cursor.execute("CREATE DATABASE IF NOT EXISTS hive_tutorial")
cursor.execute("USE hive_tutorial")

conn.commit()
cursor.execute("SHOW DATABASES")
conn.commit()
print(cursor.fetchall())

[('default',), ('hive_tutorial',)]


In [6]:
cursor.execute("DROP table world_cities")
query = """
CREATE EXTERNAL TABLE IF NOT EXISTS world_cities (
        name STRING,
        country STRING,
        subcountry STRING,
        geonameid INT)
    COMMENT 'database with information about world cities'
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    location 'hdfs://hive-hdfs/datasets/world_cities/'
    tblproperties ("skip.header.line.count"="1")
"""


cursor.execute(query)
conn.commit()

In [7]:
cursor.execute("SHOW TABLES")
conn.commit()
print(cursor.fetchall())

[('world_cities',)]


In [8]:
cursor.execute("SELECT * FROM world_cities limit 100")
conn.commit()
size = 10
records = cursor.fetchmany(size)
print("Fetching Total ", size," rows")
print("Printing each row")
for row in records:
    print(row)
cursor.close()

Fetching Total  10  rows
Printing each row
('les Escaldes', 'Andorra', 'Escaldes-Engordany', 3040051)
('Andorra la Vella', 'Andorra', 'Andorra la Vella', 3041563)
('Umm al Qaywayn', 'United Arab Emirates', 'Umm al Qaywayn', 290594)
('Ras al-Khaimah', 'United Arab Emirates', 'Raʼs al Khaymah', 291074)
('Khawr Fakkān', 'United Arab Emirates', 'Ash Shāriqah', 291696)
('Dubai', 'United Arab Emirates', 'Dubai', 292223)
('Dibba Al-Fujairah', 'United Arab Emirates', 'Al Fujayrah', 292231)
('Dibba Al-Hisn', 'United Arab Emirates', 'Al Fujayrah', 292239)
('Sharjah', 'United Arab Emirates', 'Ash Shāriqah', 292672)
('Ar Ruways', 'United Arab Emirates', 'Abu Dhabi', 292688)


In [9]:
cursor.execute("SELECT name FROM world_cities")
conn.commit()
size = 10
records = cursor.fetchmany(size)
print("Fetching Total ", size," rows")
print("Printing each row")
for row in records:
    print(row)
cursor.close()

Fetching Total  10  rows
Printing each row
('les Escaldes',)
('Andorra la Vella',)
('Umm al Qaywayn',)
('Ras al-Khaimah',)
('Khawr Fakkān',)
('Dubai',)
('Dibba Al-Fujairah',)
('Dibba Al-Hisn',)
('Sharjah',)
('Ar Ruways',)
